In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import keras
from keras import models
from keras import layers
from keras.optimizers import Adam
from keras.losses import MeanSquaredError

2023-03-16 17:33:33.762164: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [49]:
def obtain_reduced_dataset(all_features, features_to_be_used):
    feature_names = ['air_time',
                     'disp_index',
                     'gmrt_in_air',
                     'gmrt_on_paper',
                     'max_x_extension',
                     'max_y_extension',
                     'mean_acc_in_air',
                     'mean_acc_on_paper',
                     'mean_gmrt',
                     'mean_jerk_in_air',
                     'mean_jerk_on_paper',
                     'mean_speed_in_air',
                     'mean_speed_on_paper',
                     'num_of_pendown',
                     'paper_time',
                     'pressure_mean',
                     'pressure_var',
                     'total_time']
    feature_names_to_be_used = []

    for feature_num in features_to_be_used:
        for feature_name in feature_names:
            feature_names_to_be_used.append(feature_name + feature_num)
    # print(feature_names_to_be_used)

    reduced_features = all_features.filter(feature_names_to_be_used)
    # print(reduced_features)
    return reduced_features

In [50]:
# read in the dataset
dataset = pd.read_csv("DARWIN.csv")
# print(dataset)
# print(dataset.info())

In [51]:
dataset

,ID,air_time1,disp_index1,gmrt_in_air1,gmrt_on_paper1,max_x_extension1,max_y_extension1,mean_acc_in_air1,mean_acc_on_paper1,mean_gmrt1,...,mean_jerk_in_air25,mean_jerk_on_paper25,mean_speed_in_air25,mean_speed_on_paper25,num_of_pendown25,paper_time25,pressure_mean25,pressure_var25,total_time25,class
0,id_1,5160,0.000013,120.804174,86.853334,957,6601,0.361800,0.217459,103.828754,...,0.141434,0.024471,5.596487,3.184589,71,40120,1749.278166,296102.7676,144605,P
1,id_2,51980,0.000016,115.318238,83.448681,1694,6998,0.272513,0.144880,99.383459,...,0.049663,0.018368,1.665973,0.950249,129,126700,1504.768272,278744.2850,298640,P
2,id_3,2600,0.000010,229.933997,172.761858,2333,5802,0.387020,0.181342,201.347928,...,0.178194,0.017174,4.000781,2.392521,74,45480,1431.443492,144411.7055,79025,P
3,id_4,2130,0.000010,369.403342,183.193104,1756,8159,0.556879,0.164502,276.298223,...,0.113905,0.019860,4.206746,1.613522,123,67945,1465.843329,230184.7154,181220,P
4,id_5,2310,0.000007,257.997131,111.275889,987,4732,0.266077,0.145104,184.636510,...,0.121782,0.020872,3.319036,1.680629,92,37285,1841.702561,158290.0255,72575,P
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,id_170,2930,0.000010,241.736477,176.115957,1839,6439,0.253347,0.174663,208.926217,...,0.119152,0.020909,4.508709,2.233198,96,44545,1798.923336,247448.3108,80335,H
170,id_171,2140,0.000009,274.728964,234.495802,2053,8487,0.225537,0.174920,254.612383,...,0.174495,0.017640,4.685573,2.806888,84,37560,1725.619941,160664.6464,345835,H
171,id_172,3830,0.000008,151.536989,171.104693,1287,7352,0.165480,0.161058,161.320841,...,0.114472,0.017194,3.493815,2.510601,88,51675,1915.573488,128727.1241,83445,H
172,id_173,1760,0.000008,289.518195,196.411138,1674,6946,0.518937,0.202613,242.964666,...,0.114472,0.017194,3.493815,2.510601,88,51675,1915.573488,128727.1241,83445,H


In [52]:
# convert ids into number format
dataset["ID"] = dataset["ID"].apply(lambda x: int(x.split("id_")[1]))

In [53]:
#Converting Class Label P = Patient, H = Healthy -> P = 1 or has disease, H = 0 or no disease
dataset["class"] = dataset["class"].map({'P':1, 'H':0})

In [54]:
dataset

,ID,air_time1,disp_index1,gmrt_in_air1,gmrt_on_paper1,max_x_extension1,max_y_extension1,mean_acc_in_air1,mean_acc_on_paper1,mean_gmrt1,...,mean_jerk_in_air25,mean_jerk_on_paper25,mean_speed_in_air25,mean_speed_on_paper25,num_of_pendown25,paper_time25,pressure_mean25,pressure_var25,total_time25,class
0,1,5160,0.000013,120.804174,86.853334,957,6601,0.361800,0.217459,103.828754,...,0.141434,0.024471,5.596487,3.184589,71,40120,1749.278166,296102.7676,144605,1
1,2,51980,0.000016,115.318238,83.448681,1694,6998,0.272513,0.144880,99.383459,...,0.049663,0.018368,1.665973,0.950249,129,126700,1504.768272,278744.2850,298640,1
2,3,2600,0.000010,229.933997,172.761858,2333,5802,0.387020,0.181342,201.347928,...,0.178194,0.017174,4.000781,2.392521,74,45480,1431.443492,144411.7055,79025,1
3,4,2130,0.000010,369.403342,183.193104,1756,8159,0.556879,0.164502,276.298223,...,0.113905,0.019860,4.206746,1.613522,123,67945,1465.843329,230184.7154,181220,1
4,5,2310,0.000007,257.997131,111.275889,987,4732,0.266077,0.145104,184.636510,...,0.121782,0.020872,3.319036,1.680629,92,37285,1841.702561,158290.0255,72575,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,170,2930,0.000010,241.736477,176.115957,1839,6439,0.253347,0.174663,208.926217,...,0.119152,0.020909,4.508709,2.233198,96,44545,1798.923336,247448.3108,80335,0
170,171,2140,0.000009,274.728964,234.495802,2053,8487,0.225537,0.174920,254.612383,...,0.174495,0.017640,4.685573,2.806888,84,37560,1725.619941,160664.6464,345835,0
171,172,3830,0.000008,151.536989,171.104693,1287,7352,0.165480,0.161058,161.320841,...,0.114472,0.017194,3.493815,2.510601,88,51675,1915.573488,128727.1241,83445,0
172,173,1760,0.000008,289.518195,196.411138,1674,6946,0.518937,0.202613,242.964666,...,0.114472,0.017194,3.493815,2.510601,88,51675,1915.573488,128727.1241,83445,0


In [55]:
# split into features (451 columns) and labels
features = dataset.loc[:, dataset.columns != "class"]
labels = dataset["class"]
# print(features)
# print(labels)

In [56]:
dataset

,ID,air_time1,disp_index1,gmrt_in_air1,gmrt_on_paper1,max_x_extension1,max_y_extension1,mean_acc_in_air1,mean_acc_on_paper1,mean_gmrt1,...,mean_jerk_in_air25,mean_jerk_on_paper25,mean_speed_in_air25,mean_speed_on_paper25,num_of_pendown25,paper_time25,pressure_mean25,pressure_var25,total_time25,class
0,1,5160,0.000013,120.804174,86.853334,957,6601,0.361800,0.217459,103.828754,...,0.141434,0.024471,5.596487,3.184589,71,40120,1749.278166,296102.7676,144605,1
1,2,51980,0.000016,115.318238,83.448681,1694,6998,0.272513,0.144880,99.383459,...,0.049663,0.018368,1.665973,0.950249,129,126700,1504.768272,278744.2850,298640,1
2,3,2600,0.000010,229.933997,172.761858,2333,5802,0.387020,0.181342,201.347928,...,0.178194,0.017174,4.000781,2.392521,74,45480,1431.443492,144411.7055,79025,1
3,4,2130,0.000010,369.403342,183.193104,1756,8159,0.556879,0.164502,276.298223,...,0.113905,0.019860,4.206746,1.613522,123,67945,1465.843329,230184.7154,181220,1
4,5,2310,0.000007,257.997131,111.275889,987,4732,0.266077,0.145104,184.636510,...,0.121782,0.020872,3.319036,1.680629,92,37285,1841.702561,158290.0255,72575,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,170,2930,0.000010,241.736477,176.115957,1839,6439,0.253347,0.174663,208.926217,...,0.119152,0.020909,4.508709,2.233198,96,44545,1798.923336,247448.3108,80335,0
170,171,2140,0.000009,274.728964,234.495802,2053,8487,0.225537,0.174920,254.612383,...,0.174495,0.017640,4.685573,2.806888,84,37560,1725.619941,160664.6464,345835,0
171,172,3830,0.000008,151.536989,171.104693,1287,7352,0.165480,0.161058,161.320841,...,0.114472,0.017194,3.493815,2.510601,88,51675,1915.573488,128727.1241,83445,0
172,173,1760,0.000008,289.518195,196.411138,1674,6946,0.518937,0.202613,242.964666,...,0.114472,0.017194,3.493815,2.510601,88,51675,1915.573488,128727.1241,83445,0


In [57]:
# reduce features
reduced_features = features
reduced_features_input = int(input("Would you like to reduce the features (0 for no, 1 for yes): "))
reduced_features_to_be_used = ['17', '19', '21', '22', '23']
if reduced_features_input:
    reduced_features = obtain_reduced_dataset(features, reduced_features_to_be_used)

Would you like to reduce the features (0 for no, 1 for yes): 1


In [58]:
features

,ID,air_time1,disp_index1,gmrt_in_air1,gmrt_on_paper1,max_x_extension1,max_y_extension1,mean_acc_in_air1,mean_acc_on_paper1,mean_gmrt1,...,mean_gmrt25,mean_jerk_in_air25,mean_jerk_on_paper25,mean_speed_in_air25,mean_speed_on_paper25,num_of_pendown25,paper_time25,pressure_mean25,pressure_var25,total_time25
0,1,5160,0.000013,120.804174,86.853334,957,6601,0.361800,0.217459,103.828754,...,249.729085,0.141434,0.024471,5.596487,3.184589,71,40120,1749.278166,296102.7676,144605
1,2,51980,0.000016,115.318238,83.448681,1694,6998,0.272513,0.144880,99.383459,...,77.258394,0.049663,0.018368,1.665973,0.950249,129,126700,1504.768272,278744.2850,298640
2,3,2600,0.000010,229.933997,172.761858,2333,5802,0.387020,0.181342,201.347928,...,193.667018,0.178194,0.017174,4.000781,2.392521,74,45480,1431.443492,144411.7055,79025
3,4,2130,0.000010,369.403342,183.193104,1756,8159,0.556879,0.164502,276.298223,...,163.065803,0.113905,0.019860,4.206746,1.613522,123,67945,1465.843329,230184.7154,181220
4,5,2310,0.000007,257.997131,111.275889,987,4732,0.266077,0.145104,184.636510,...,147.094679,0.121782,0.020872,3.319036,1.680629,92,37285,1841.702561,158290.0255,72575
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,170,2930,0.000010,241.736477,176.115957,1839,6439,0.253347,0.174663,208.926217,...,200.568126,0.119152,0.020909,4.508709,2.233198,96,44545,1798.923336,247448.3108,80335
170,171,2140,0.000009,274.728964,234.495802,2053,8487,0.225537,0.174920,254.612383,...,231.243565,0.174495,0.017640,4.685573,2.806888,84,37560,1725.619941,160664.6464,345835
171,172,3830,0.000008,151.536989,171.104693,1287,7352,0.165480,0.161058,161.320841,...,177.935566,0.114472,0.017194,3.493815,2.510601,88,51675,1915.573488,128727.1241,83445
172,173,1760,0.000008,289.518195,196.411138,1674,6946,0.518937,0.202613,242.964666,...,177.935566,0.114472,0.017194,3.493815,2.510601,88,51675,1915.573488,128727.1241,83445


In [59]:
reduced_features

,air_time17,disp_index17,gmrt_in_air17,gmrt_on_paper17,max_x_extension17,max_y_extension17,mean_acc_in_air17,mean_acc_on_paper17,mean_gmrt17,mean_jerk_in_air17,...,mean_gmrt23,mean_jerk_in_air23,mean_jerk_on_paper23,mean_speed_in_air23,mean_speed_on_paper23,num_of_pendown23,paper_time23,pressure_mean23,pressure_var23,total_time23
0,26660,0.000025,233.520832,287.757539,11411,14027,1.020258,0.184635,260.639185,0.192499,...,158.408585,0.024301,0.020756,2.453629,3.183865,12,5195,1665.775746,314371.4367,16160
1,54370,0.000030,114.279952,85.633430,11719,12762,0.572728,0.172266,99.956691,0.098941,...,69.443302,0.041392,0.016741,1.806707,0.782637,14,15240,1597.799213,257556.6467,29900
2,27640,0.000029,355.428038,166.664326,12885,13413,2.116709,0.137683,261.046182,0.410326,...,153.723730,0.031521,0.018083,2.415460,2.730515,12,6535,1599.237184,264722.8435,13865
3,27000,0.000026,262.309253,166.014262,11690,13895,1.060545,0.161015,214.161758,0.196077,...,188.826738,0.031505,0.017185,4.231579,2.666430,11,6380,1493.021944,252209.4556,13585
4,22375,0.000019,318.563434,206.807610,11746,12424,1.732136,0.174221,262.685522,0.330048,...,131.080540,0.022273,0.018069,1.873943,2.154279,12,4805,1713.592092,260030.9283,10145
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,12595,0.000027,341.258753,288.192087,11962,14238,1.544344,0.154135,314.725420,0.292945,...,221.568625,0.034484,0.023558,4.823167,3.163325,13,5105,1825.012733,219916.2457,7745
170,13250,0.000031,360.743997,228.464449,12177,13644,1.616910,0.125823,294.604223,0.310986,...,266.271081,0.046389,0.017801,4.254971,4.349164,12,4215,1767.243179,138846.4830,8025
171,13250,0.000031,360.743997,228.464449,12177,13644,1.616910,0.125823,294.604223,0.310986,...,208.384600,0.033345,0.016877,4.927189,2.492176,12,5910,1930.645516,128484.0156,7760
172,13250,0.000031,360.743997,228.464449,12177,13644,1.616910,0.125823,294.604223,0.310986,...,208.384600,0.033345,0.016877,4.927189,2.492176,12,5910,1930.645516,128484.0156,7760


In [60]:
features_noID = dataset.drop(labels={'ID', 'class'}, axis=1)
features_noID

,air_time1,disp_index1,gmrt_in_air1,gmrt_on_paper1,max_x_extension1,max_y_extension1,mean_acc_in_air1,mean_acc_on_paper1,mean_gmrt1,mean_jerk_in_air1,...,mean_gmrt25,mean_jerk_in_air25,mean_jerk_on_paper25,mean_speed_in_air25,mean_speed_on_paper25,num_of_pendown25,paper_time25,pressure_mean25,pressure_var25,total_time25
0,5160,0.000013,120.804174,86.853334,957,6601,0.361800,0.217459,103.828754,0.051836,...,249.729085,0.141434,0.024471,5.596487,3.184589,71,40120,1749.278166,296102.7676,144605
1,51980,0.000016,115.318238,83.448681,1694,6998,0.272513,0.144880,99.383459,0.039827,...,77.258394,0.049663,0.018368,1.665973,0.950249,129,126700,1504.768272,278744.2850,298640
2,2600,0.000010,229.933997,172.761858,2333,5802,0.387020,0.181342,201.347928,0.064220,...,193.667018,0.178194,0.017174,4.000781,2.392521,74,45480,1431.443492,144411.7055,79025
3,2130,0.000010,369.403342,183.193104,1756,8159,0.556879,0.164502,276.298223,0.090408,...,163.065803,0.113905,0.019860,4.206746,1.613522,123,67945,1465.843329,230184.7154,181220
4,2310,0.000007,257.997131,111.275889,987,4732,0.266077,0.145104,184.636510,0.037528,...,147.094679,0.121782,0.020872,3.319036,1.680629,92,37285,1841.702561,158290.0255,72575
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,2930,0.000010,241.736477,176.115957,1839,6439,0.253347,0.174663,208.926217,0.032691,...,200.568126,0.119152,0.020909,4.508709,2.233198,96,44545,1798.923336,247448.3108,80335
170,2140,0.000009,274.728964,234.495802,2053,8487,0.225537,0.174920,254.612383,0.032059,...,231.243565,0.174495,0.017640,4.685573,2.806888,84,37560,1725.619941,160664.6464,345835
171,3830,0.000008,151.536989,171.104693,1287,7352,0.165480,0.161058,161.320841,0.022705,...,177.935566,0.114472,0.017194,3.493815,2.510601,88,51675,1915.573488,128727.1241,83445
172,1760,0.000008,289.518195,196.411138,1674,6946,0.518937,0.202613,242.964666,0.090686,...,177.935566,0.114472,0.017194,3.493815,2.510601,88,51675,1915.573488,128727.1241,83445


In [61]:
print(dataset.iloc[0])

ID                      1.000000
air_time1            5160.000000
disp_index1             0.000013
gmrt_in_air1          120.804174
gmrt_on_paper1         86.853334
                       ...      
paper_time25        40120.000000
pressure_mean25      1749.278166
pressure_var25     296102.767600
total_time25       144605.000000
class                   1.000000
Name: 0, Length: 452, dtype: float64


In [62]:
print(features_noID.iloc[0])

air_time1             5160.000000
disp_index1              0.000013
gmrt_in_air1           120.804174
gmrt_on_paper1          86.853334
max_x_extension1       957.000000
                        ...      
num_of_pendown25        71.000000
paper_time25         40120.000000
pressure_mean25       1749.278166
pressure_var25      296102.767600
total_time25        144605.000000
Name: 0, Length: 450, dtype: float64


In [63]:
#FULL DATASET
# split into train test
x_train, x_test, y_train, y_test = train_test_split(features_noID, labels, test_size=0.2, random_state=42)
# print(x_train)
# print(x_test)

In [64]:
x_train

,air_time1,disp_index1,gmrt_in_air1,gmrt_on_paper1,max_x_extension1,max_y_extension1,mean_acc_in_air1,mean_acc_on_paper1,mean_gmrt1,mean_jerk_in_air1,...,mean_gmrt25,mean_jerk_in_air25,mean_jerk_on_paper25,mean_speed_in_air25,mean_speed_on_paper25,num_of_pendown25,paper_time25,pressure_mean25,pressure_var25,total_time25
118,540,0.000007,557.998222,221.396848,1697,6343,0.231201,0.157785,389.697535,0.037565,...,244.045421,0.202010,0.019898,5.741075,2.535960,73,31590,1942.588003,97106.70791,55120
142,1800,0.000008,254.998848,169.725972,1476,7247,0.203489,0.153118,212.362410,0.030854,...,180.306786,0.106158,0.020339,3.220606,2.324440,103,28280,1674.497878,224182.68560,61885
163,2740,0.000009,354.518156,289.414055,1831,9729,0.311624,0.216704,321.966106,0.041679,...,188.204705,0.113037,0.017907,3.281408,2.635369,110,55115,1648.218997,219329.47010,124760
38,2635,0.000007,261.166647,147.334293,2099,3936,0.620047,0.149459,204.250470,0.112002,...,225.085577,0.091910,0.022776,4.458446,3.211986,93,27320,1491.778917,235001.24650,50155
139,2975,0.000009,396.225929,153.497310,1628,7466,0.602640,0.148833,274.861619,0.099270,...,407.395427,0.375078,0.022148,8.964604,5.292562,68,27560,1718.044086,142453.21450,37840
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,2615,0.000014,637.692285,228.919963,2105,13749,0.268277,0.155770,433.306124,0.032642,...,263.058996,0.167926,0.026026,5.473425,3.674722,98,30115,1939.105761,101814.91310,75615
106,2380,0.000007,150.688702,170.070042,1872,4892,0.408521,0.162480,160.379372,0.071124,...,187.005105,0.089351,0.019781,3.431968,2.628899,106,46350,1561.181230,292377.27850,97400
14,2900,0.000011,305.092663,130.501402,1214,8202,0.979940,0.127559,217.797032,0.185118,...,105.215372,0.071264,0.017608,1.844563,1.505821,84,55645,874.566718,153968.14580,117830
92,5395,0.000013,152.234441,133.638302,1939,9778,0.191476,0.157144,142.936372,0.023057,...,284.323967,0.204001,0.020445,5.156128,3.776394,61,22735,1850.260391,154122.41430,53235


In [65]:
y_train

118    0
142    0
163    0
38     1
139    0
      ..
71     1
106    0
14     1
92     0
102    0
Name: class, Length: 139, dtype: int64

In [86]:
model = models.Sequential()
model.add(layers.Dense(450, input_shape=(450,), activation='relu'))
#model.add(layers.Dense(250, activation='relu'))
#model.add(layers.Dense(150, activation='relu'))
model.add(layers.Dense(50, activation='relu'))
#model.add(layers.Dense(25, activation='relu'))
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [87]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [88]:
model.fit(x_train, y_train, epochs=30, batch_size=16)

Epoch 1/30
9/9 [==============================] - 0s 3ms/step - loss: 7243.8789 - accuracy: 0.5180
Epoch 2/30
9/9 [==============================] - 0s 2ms/step - loss: 1590.6716 - accuracy: 0.7554
Epoch 3/30
9/9 [==============================] - 0s 2ms/step - loss: 1262.6135 - accuracy: 0.8489
Epoch 4/30
9/9 [==============================] - 0s 2ms/step - loss: 417.2362 - accuracy: 0.8921
Epoch 5/30
9/9 [==============================] - 0s 2ms/step - loss: 416.0245 - accuracy: 0.9137
Epoch 6/30
9/9 [==============================] - 0s 2ms/step - loss: 131.6293 - accuracy: 0.9640
Epoch 7/30
9/9 [==============================] - 0s 2ms/step - loss: 113.4951 - accuracy: 0.9353
Epoch 8/30
9/9 [==============================] - 0s 2ms/step - loss: 203.9039 - accuracy: 0.9496
Epoch 9/30
9/9 [==============================] - 0s 2ms/step - loss: 256.8310 - accuracy: 0.9209
Epoch 10/30
9/9 [==============================] - 0s 2ms/step - loss: 69.7535 - accuracy: 0.9568
Epoch 11/30
9/9 [

In [89]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print(test_loss, test_acc)

2/2 [==============================] - 0s 3ms/step - loss: 564.7165 - accuracy: 0.8857
564.7164916992188 0.8857142925262451


In [90]:
# get predictions
y_pred = model.predict(x_test)
# print(y_test.to_numpy())
# print(y_pred)

2/2 [==============================] - 0s 2ms/step


In [91]:
# get confusion matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

# calculate accuracy, specificity, & sensitivity
accuracy = test_acc
specificity = tn / (tn + fp)
sensitivity = tp / (tp + fn)

# report results
#if reduced_features_input:
#    print("Results on Feature set (" + ','.join(reduced_features_to_be_used) + "): ")
#else:
print("Results on Entire Feature set: ")
print("     Neural Network: ", accuracy)
print("     Neural Network specificity: ", specificity)
print("     Neural Network sensitivity: ", sensitivity)

Results on Entire Feature set: 
     Neural Network:  0.8857142925262451
     Neural Network specificity:  0.8666666666666667
     Neural Network sensitivity:  0.9


-

In [92]:
#REDUCED DATASET
# split into train test
x_train, x_test, y_train, y_test = train_test_split(reduced_features, labels, test_size=0.2, random_state=42)
# print(x_train)
# print(x_test)

In [93]:
x_train

,air_time17,disp_index17,gmrt_in_air17,gmrt_on_paper17,max_x_extension17,max_y_extension17,mean_acc_in_air17,mean_acc_on_paper17,mean_gmrt17,mean_jerk_in_air17,...,mean_gmrt23,mean_jerk_in_air23,mean_jerk_on_paper23,mean_speed_in_air23,mean_speed_on_paper23,num_of_pendown23,paper_time23,pressure_mean23,pressure_var23,total_time23
118,43530,0.000021,94.880298,218.767818,11344,13113,0.432612,0.172679,156.824058,0.077091,...,238.133566,0.029446,0.019727,4.776750,3.705256,11,3405,1912.208517,129527.6070,5855
142,24050,0.000023,229.192838,209.845763,11808,12796,1.289585,0.144842,219.519300,0.241694,...,179.288556,0.024953,0.018883,3.015706,2.908564,11,3440,1794.031977,211693.4728,6205
163,45270,0.000025,130.733081,187.156593,11835,14154,0.600844,0.173565,158.944837,0.104022,...,135.637596,0.037666,0.017050,2.279671,2.344970,13,6850,1759.447445,215165.5450,15290
38,8800,0.000026,409.070836,253.186273,12262,13480,1.495398,0.166840,331.128555,0.282904,...,147.882433,0.026129,0.018262,2.925066,2.093291,14,7490,1827.065421,182517.9450,14875
139,1900,0.000003,246.678577,188.072600,1093,2926,0.218801,0.154427,217.375588,0.024378,...,256.060104,0.081006,0.019493,5.338402,3.944983,11,4500,1782.427778,140623.6470,7010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,80560,0.000029,116.692547,178.123760,11986,13820,0.483837,0.137423,147.408153,0.080809,...,101.873362,0.022582,0.021640,1.384106,2.111713,12,5245,1721.736892,189621.1434,13655
106,8675,0.000029,451.524237,323.921244,12758,14985,1.900812,0.173330,387.722741,0.363411,...,213.711854,0.030532,0.016306,4.239107,3.008462,11,5080,1565.994094,232361.9173,7835
14,48370,0.000032,85.790213,128.821222,11912,14434,0.507063,0.161046,107.305718,0.084445,...,86.323181,0.032748,0.019121,1.564581,1.302347,13,11235,929.834891,211744.6443,22445
92,5460,0.000020,617.616701,344.014717,12067,14238,3.346877,0.169224,480.815709,0.659471,...,218.908488,0.026147,0.024485,2.457211,4.678430,11,2540,1696.688976,190703.0332,8310


In [94]:
y_train

118    0
142    0
163    0
38     1
139    0
      ..
71     1
106    0
14     1
92     0
102    0
Name: class, Length: 139, dtype: int64

In [127]:
model = models.Sequential()
model.add(layers.Dense(90, input_shape=(90,), activation='relu'))
model.add(layers.Dense(50, activation='relu'))
#model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [128]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [129]:
model.fit(x_train, y_train, epochs=30, batch_size=16)

Epoch 1/30
9/9 [==============================] - 0s 2ms/step - loss: 6537.0171 - accuracy: 0.5108
Epoch 2/30
9/9 [==============================] - 0s 1ms/step - loss: 2000.5157 - accuracy: 0.6259
Epoch 3/30
9/9 [==============================] - 0s 2ms/step - loss: 1251.6628 - accuracy: 0.6331
Epoch 4/30
9/9 [==============================] - 0s 3ms/step - loss: 559.8530 - accuracy: 0.7194
Epoch 5/30
9/9 [==============================] - 0s 2ms/step - loss: 457.9430 - accuracy: 0.8201
Epoch 6/30
9/9 [==============================] - 0s 2ms/step - loss: 463.2213 - accuracy: 0.7842
Epoch 7/30
9/9 [==============================] - 0s 2ms/step - loss: 440.4888 - accuracy: 0.7986
Epoch 8/30
9/9 [==============================] - 0s 2ms/step - loss: 185.6464 - accuracy: 0.8705
Epoch 9/30
9/9 [==============================] - 0s 2ms/step - loss: 246.3769 - accuracy: 0.8129
Epoch 10/30
9/9 [==============================] - 0s 2ms/step - loss: 200.5767 - accuracy: 0.8561
Epoch 11/30
9/9 

In [130]:
test_loss, test_acc = model.evaluate(x_test, y_test)

2/2 [==============================] - 0s 3ms/step - loss: 395.7905 - accuracy: 0.8000


In [131]:
# get predictions
y_pred = model.predict(x_test)
# print(y_test.to_numpy())
# print(y_pred)

2/2 [==============================] - 0s 3ms/step


In [135]:
# get confusion matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

# calculate accuracy, specificity, & sensitivity
accuracy = test_acc
specificity = tn / (tn + fp)
sensitivity = tp / (tp + fn)

# report results
print("Results on Feature set (" + ', '.join(reduced_features_to_be_used) + "): ")
print("     Neural Network: ", accuracy)
print("     Neural Network specificity: ", specificity)
print("     Neural Network sensitivity: ", sensitivity)

Results on Feature set (17, 19, 21, 22, 23): 
     Neural Network:  0.800000011920929
     Neural Network specificity:  0.9333333333333333
     Neural Network sensitivity:  0.7
